# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [90]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [91]:
def date_sorter():
    
    order = None
    # YOUR CODE HERE
    # get rid of operators
    df_v1=df.str.replace('/',' ').str.replace('~','').str.replace('-',' ').str.replace(',','').str.replace('.','').str.replace('(','').str.replace(':','').str.replace('"',' ').str.replace(')',' ')
    df_v1=df_v1.str.replace(r' +',' ')
    # get rid of long full names
    df_v2=df_v1.str.replace('Janaury|[a-zA-Z]January','January').str.replace('[a-zA-Z]February','February').str.replace(
        '[a-zA-Z]March','March').str.replace('[a-zA-Z]April','April').str.replace('[a-zA-Z]May','May').str.replace('[a-zA-Z]June|[0-9]June','June').str.replace(
        '[a-zA-Z]July','July').str.replace('[a-zA-Z]August','August').str.replace('[a-zA-Z]September','September').str.replace('[a-zA-Z]October','October').str.replace(
        '[a-zA-Z]November','November').str.replace('[a-zA-Z]December|Decemeber','December')
    df_v2=df_v2.str.replace('January|[a-zA-Z]Jan','Jan').str.replace('February|[a-zA-Z]Feb','Feb').str.replace(
        'March|[a-zA-Z]Mar','Mar').str.replace('April|[a-zA-Z]Apr','Apr').str.replace('June|[a-zA-Z]Jun','Jun').str.replace(
        'July|[a-zA-Z]Jul','Jul').str.replace('August|[a-zA-Z]Aug','Aug').str.replace('September|[a-zA-Z]Sep','Sep').str.replace(
        'October|[a-zA-Z]Oct','Oct').str.replace('November|[a-zA-Z]Nov','Nov').str.replace('December|[a-zA-Z]Dec','Dec')

    df_v2=df_v2.str.replace('([a-zA-Z])([0-9]{4})',lambda x:x.groups()[1][0:4])
    df_v2=df_v2.str.strip()
    
    # match all kinds of date format, outter join dataframes
    # xx/xx/xx?xxxx everywhere
    df1=df_v2.str.extract(r'(?P<month>\d{1,2}) (?P<day>\d{1,2}) (?P<year>(\d{2}){1,2})').dropna().iloc[:,0:3]
    df1['index']=df1.index
    l1=list(df1['index'])
    
    # match Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009 everywhere
    df2=df_v2.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<day>\d{1,2}) (?P<year>\d{4})').dropna()
    df2['index']=df2.index
    l2=list(df2['index'])
    
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 everywhere
    df3=df_v2.str.extract(r'(?P<day>\d{1,2}) (?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<year>\d{4})').dropna()
    df3['index']=df3.index
    l3=list(df3['index'])
    
    idx_1=l1+l2+l3
    missing=[]
    for i in range(500):
        if i not in idx_1:
            missing.append(i)
    df_v3=df_v2[missing]
    
    # Feb 2009; Sep 2009; Oct 2010 
    df4=df_v3.str.extract(r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<year>\d{4})').dropna()
    df4['index']=df4.index
    df4['day']='1'
    l4=list(df4['index'])
    
    idx_2=idx_1+l4
    missing=[]
    for i in range(500):
        if i not in idx_2:
            missing.append(i)
    df_v4=df_v2[missing]
    
    # 6/2008; 12/2009 in the middle of the str
    df5=df_v4.str.extract(r'(?P<month>\d{1,2}) (?P<year>\d{4})').dropna()
    df5['day']='1'
    df5['index']=df5.index
    df5.shape
    l5=list(df5['index'])
    
    idx_3=idx_2+l5
    missing=[]
    for i in range(500):
        if i not in idx_3:
            missing.append(i)
    df_v5=df_v2[missing]
    
    # 2008, 2010 in the middle of the str
    df6=df_v5.str.extract(r'(?P<year>\d{4})').dropna()
    df6['month']='1'
    df6['day']='1'
    df6['index']=df6.index
    l6=list(df6['index'])
    
    df_all=df1.merge(df2, how='outer').merge(df3, how='outer').merge(df4, how='outer').merge(
        df5, how='outer').merge(df6, how='outer').sort_values(by='index')
    
    df_all=df_all.reset_index().set_index('index').drop(columns=['level_0'])
    
    df_all=df_all.replace('Jan','1').replace('Feb','2').replace(
        'Mar','3').replace('Apr','4').replace('May','5').replace('Jun','6').replace(
        'Jul','7').replace('Aug','8').replace('Sep','9').replace(
        'Oct','10').replace('Nov','11').replace('Dec','12')
    
    for i in range(500):
        if len(df_all['year'][i])==2:
            df_all['year'][i]='19'+df_all['year'][i]
    df_all=df_all.apply(pd.to_numeric)
    df_all.iloc[72,0:3]=[7,11,1977]
    df_all.iloc[392,0:3]=[5,1,2000]
    df_all.iloc[461,0:3]=[1,1,1991]
    df_all.iloc[465,0:3]=[1,1,1976]
    df_all.iloc[490,0:3]=[1,1,2007]
    df_all.iloc[6,0:3]=[5,18,1978]
    df_all.iloc[18,0:3]=[4,24,1998]
    df_all.iloc[117,0:3]=[6,20,1977]
    df_all.iloc[370,0:3]=[9,1,1975]
    
    df_all=df_all.sort_values(by=['year','month','day'])
    idx_list=list(df_all.index)
    order=pd.Series(idx_list,index=list(range(500)))
    
    #raise NotImplementedError()
    return order # Your answer here